In [14]:
from __future__ import division 
import numpy as np
import pandas as pd
import scipy
from scipy.stats import pearsonr
import scipy.stats as stat
import scipy.signal as sps
import scipy.io.wavfile as wf
from vmdpy import VMD
import glob
import os
import librosa as lib
from librosa import feature as libf
from spafe.features.gfcc import gfcc

import augly.audio as audaugs
import matplotlib.pyplot as plt 
import soundfile as sf
from gammatone import gtgram

from scipy import fft, arange
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile

import os
import glob as glob

import pywt
import pywt.data

from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

VMD 

In [20]:
alpha = 2000        
tau = 0.              
#K = 4            
DC = 0              
init = 1           
tol = 1e-7
sample_rate=4000
sr=4000
fs=30000

In [21]:
def time_domain_features(y):
    return np.mean(y), np.std(y), np.sqrt(np.mean(y**2)), np.sqrt(np.sum(y**2)), np.amax(np.absolute(y), axis=0)/np.sqrt(np.mean(y**2)), np.sqrt(np.mean(y**2))/np.mean(np.absolute(y)), np.sum(y*y), np.mean(np.absolute(y - np.mean(y))), np.mean(np.absolute(y - np.median(y))), stat.skew(y), stat.kurtosis(y), np.max(y)-np.min(y), np.log(np.sum(y*y)), np.max(y)-np.abs(np.sqrt(np.mean(y**2)))

In [22]:
def spectral_domain_features(y):
    yf = scipy.fft.rfft(y)    
    return np.mean(np.abs(yf)), np.std(yf), stat.skew(np.abs(yf)), stat.kurtosis(np.abs(yf)), np.mean(libf.spectral_centroid(y)), np.mean(libf.spectral_rolloff(y)), np.mean(libf.spectral_flatness(y))

In [23]:
def frequency_spectrum(x, sf):
    
    x = x - np.average(x)  # zero-centering

    n = len(x)
    k = arange(n)
    tarr = n / float(sf)
    frqarr = k / float(tarr)  # two sides frequency range

    frqarr = frqarr[range(n // 2)]  # one side frequency range

    x = fft.fft(x) / n  # fft computing and normalization
    x = x[range(n // 2)]

    return frqarr, abs(x) 

In [24]:
def get_gfcc(y):
    fq, x=frequency_spectrum(y, sr)
    h_f=0
    h_a=0
    for j in range(fq.shape[0]):
        if(h_a<x[j]):
            h_f=fq[j]
            h_a=x[j]
    b, a=sps.gammatone(h_f, 'iir', numtaps=560, fs=4000)
    y_filt=sps.lfilter(b, a, y)
    y_gfcc=libf.mfcc(y=y_filt, sr=sr, n_mfcc=13)
    return y_gfcc

In [25]:
def mfcc_gfcc(y):
    mfcc = libf.mfcc(y=y, sr=sr, n_mfcc=13)
    mfcc_r=[]
    for i in range(13):
        mfcc_r.append(np.mean(mfcc[i]))
    gfcc1=get_gfcc(y)
    gfcc_r=[]
    for i in range(13):
        gfcc_r.append(np.mean(gfcc1[i]))
    return mfcc_r, gfcc_r

t1: mean
t2: standard deviation
t3: root mean square
t4: root sum of squares
t5: crest factor
t6: shape factor
t7: energy
t8: mean abs deviation
t9: median abs deviation
t10: skewness
t11: kurtosis
t12: peak to peak
t13: log energy
t14: peak to rms

s1: s.mean
s2: s.standard deviation
s3: s.skewness
s4: s.kurtosis
s5: spectral centroid
s6: spectral rolloff
s7: spectral flatness

c1: mfcc
c2: gfcc

In [26]:
t_l=["mean", "standard deviation", "root mean square", "root sum of squares", "crest factor", "shape factor", "energy", "mean abs deviation", "median abs deviation", "skewness", "kurtosis", "peak to peak", "log energy", "peak to rms"]

s_l = ["s.mean", "s.standard deviation", "s.skewness", "s.kurtosis", "spectral centroid", "spectral rolloff", "spectral flatness"]

c_l=["mfcc", "gfcc"]

In [27]:
def extract_features(y):
    t = time_domain_features(y)
    t1, t2, t3, t4, t5, t6, t7, t8, t9, t10, t11, t12, t13, t14 = t

    s = spectral_domain_features(y)
    s1, s2, s3, s4, s5, s6, s7 = s

    c = mfcc_gfcc(y)
    c1, c2 = c
    
    return t, s, c

In [28]:
def vmd(y, K):
    u, u_hat, omega = VMD(y, alpha, tau, K, DC, init, tol)  
    return u, u_hat, omega

In [29]:
def Merge(dict1, dict2):
    res = {**dict1, **dict2}
    return res

In [30]:
def get_overall_features(y):
    
    #extracting features of dwt 
    
    d_w=pywt.wavedec(y, wavelet='coif5', level=16)
    
    feat_a={}

    ta, sa, ca=extract_features(d_w[0])

    for i in range(len(ta)):
        feat_a["Approx Coeff-"+str(t_l[i])]=ta[i]

    for i in range(len(sa)):
        feat_a["Approx Coeff-"+str(s_l[i])]=sa[i]

    for i in range(len(ca[0])):
        feat_a["Approx Coeff-MFCC-"+str(i+1)]=ca[0][i]
        feat_a["Approx Coeff-GFCC-"+str(i+1)]=ca[1][i]
        

    t=[]
    c=[]
    s=[]
    for i in range(1, len(d_w)):
        f=extract_features(d_w[i])
        t.append(f[0])
        s.append(f[1])
        c.append(f[2])


    feat_d={}

    for j in range(len(t[0])):
        f=[]
        for i in range(len(t)):
            f.append(t[i][j])
        feat_d["Detail Coeff-"+str(t_l[j])]=np.mean(f)
    for j in range(len(s[0])):
        f=[]
        for i in range(len(s)):
            f.append(s[i][j])
        feat_d["Detail Coeff-"+str(s_l[j])]=np.mean(f)   
    for i in range(13):
        mfcc=[]
        gfcc=[]
        for j in range(len(c)):
            mfcc.append(c[j][0][i])
            gfcc.append(c[j][1][i])
        feat_d["Detail Coeff-GFCC-"+str(i+1)]=np.mean(gfcc)
        feat_d["Detail Coeff-MFCC-"+str(i+1)]=np.mean(mfcc)

    #extracting features of vmd
    
    u, u_hat, omega=vmd(y, 4)

    t=[]
    c=[]
    s=[]
    
    for i in range(4):
        f=extract_features(u[i])
        t.append(f[0])
        s.append(f[1])
        c.append(f[2])

    feat_v={}

    for j in range(len(t[0])):
        f=[]
        for i in range(len(t)):
            f.append(t[i][j])
        feat_v["VMD-"+t_l[j]]=np.mean(f)
    for j in range(len(s[0])):
        f=[]
        for i in range(len(s)):
            f.append(s[i][j])
        feat_v["VMD-"+s_l[j]]=np.mean(f) 
    for i in range(13):
        mfcc=[]
        gfcc=[]
        for j in range(len(c)):
            mfcc.append(c[j][0][i])
            gfcc.append(c[j][1][i])
        feat_v["VMD-GFCC-"+str(i+1)]=np.mean(gfcc)
        feat_v["VMD-MFCC-"+str(i+1)]=np.mean(mfcc)
        
    feat=Merge(feat_v, feat_a)
    feat=Merge(feat, feat_d)
    
    return feat

In [33]:
keys=['VMD-mean',
 'VMD-standard deviation',
 'VMD-root mean square',
 'VMD-root sum of squares',
 'VMD-crest factor',
 'VMD-shape factor',
 'VMD-energy',
 'VMD-mean abs deviation',
 'VMD-median abs deviation',
 'VMD-skewness',
 'VMD-kurtosis',
 'VMD-peak to peak',
 'VMD-log energy',
 'VMD-peak to rms',
 'VMD-s.mean',
 'VMD-s.standard deviation',
 'VMD-s.skewness',
 'VMD-s.kurtosis',
 'VMD-spectral centroid',
 'VMD-spectral rolloff',
 'VMD-spectral flatness',
 'VMD-GFCC-1',
 'VMD-MFCC-1',
 'VMD-GFCC-2',
 'VMD-MFCC-2',
 'VMD-GFCC-3',
 'VMD-MFCC-3',
 'VMD-GFCC-4',
 'VMD-MFCC-4',
 'VMD-GFCC-5',
 'VMD-MFCC-5',
 'VMD-GFCC-6',
 'VMD-MFCC-6',
 'VMD-GFCC-7',
 'VMD-MFCC-7',
 'VMD-GFCC-8',
 'VMD-MFCC-8',
 'VMD-GFCC-9',
 'VMD-MFCC-9',
 'VMD-GFCC-10',
 'VMD-MFCC-10',
 'VMD-GFCC-11',
 'VMD-MFCC-11',
 'VMD-GFCC-12',
 'VMD-MFCC-12',
 'VMD-GFCC-13',
 'VMD-MFCC-13',
 'Approx Coeff-mean',
 'Approx Coeff-standard deviation',
 'Approx Coeff-root mean square',
 'Approx Coeff-root sum of squares',
 'Approx Coeff-crest factor',
 'Approx Coeff-shape factor',
 'Approx Coeff-energy',
 'Approx Coeff-mean abs deviation',
 'Approx Coeff-median abs deviation',
 'Approx Coeff-skewness',
 'Approx Coeff-kurtosis',
 'Approx Coeff-peak to peak',
 'Approx Coeff-log energy',
 'Approx Coeff-peak to rms',
 'Approx Coeff-s.mean',
 'Approx Coeff-s.standard deviation',
 'Approx Coeff-s.skewness',
 'Approx Coeff-s.kurtosis',
 'Approx Coeff-spectral centroid',
 'Approx Coeff-spectral rolloff',
 'Approx Coeff-spectral flatness',
 'Approx Coeff-MFCC-1',
 'Approx Coeff-GFCC-1',
 'Approx Coeff-MFCC-2',
 'Approx Coeff-GFCC-2',
 'Approx Coeff-MFCC-3',
 'Approx Coeff-GFCC-3',
 'Approx Coeff-MFCC-4',
 'Approx Coeff-GFCC-4',
 'Approx Coeff-MFCC-5',
 'Approx Coeff-GFCC-5',
 'Approx Coeff-MFCC-6',
 'Approx Coeff-GFCC-6',
 'Approx Coeff-MFCC-7',
 'Approx Coeff-GFCC-7',
 'Approx Coeff-MFCC-8',
 'Approx Coeff-GFCC-8',
 'Approx Coeff-MFCC-9',
 'Approx Coeff-GFCC-9',
 'Approx Coeff-MFCC-10',
 'Approx Coeff-GFCC-10',
 'Approx Coeff-MFCC-11',
 'Approx Coeff-GFCC-11',
 'Approx Coeff-MFCC-12',
 'Approx Coeff-GFCC-12',
 'Approx Coeff-MFCC-13',
 'Approx Coeff-GFCC-13',
 'Detail Coeff-mean',
 'Detail Coeff-standard deviation',
 'Detail Coeff-root mean square',
 'Detail Coeff-root sum of squares',
 'Detail Coeff-crest factor',
 'Detail Coeff-shape factor',
 'Detail Coeff-energy',
 'Detail Coeff-mean abs deviation',
 'Detail Coeff-median abs deviation',
 'Detail Coeff-skewness',
 'Detail Coeff-kurtosis',
 'Detail Coeff-peak to peak',
 'Detail Coeff-log energy',
 'Detail Coeff-peak to rms',
 'Detail Coeff-s.mean',
 'Detail Coeff-s.standard deviation',
 'Detail Coeff-s.skewness',
 'Detail Coeff-s.kurtosis',
 'Detail Coeff-spectral centroid',
 'Detail Coeff-spectral rolloff',
 'Detail Coeff-spectral flatness',
 'Detail Coeff-GFCC-1',
 'Detail Coeff-MFCC-1',
 'Detail Coeff-GFCC-2',
 'Detail Coeff-MFCC-2',
 'Detail Coeff-GFCC-3',
 'Detail Coeff-MFCC-3',
 'Detail Coeff-GFCC-4',
 'Detail Coeff-MFCC-4',
 'Detail Coeff-GFCC-5',
 'Detail Coeff-MFCC-5',
 'Detail Coeff-GFCC-6',
 'Detail Coeff-MFCC-6',
 'Detail Coeff-GFCC-7',
 'Detail Coeff-MFCC-7',
 'Detail Coeff-GFCC-8',
 'Detail Coeff-MFCC-8',
 'Detail Coeff-GFCC-9',
 'Detail Coeff-MFCC-9',
 'Detail Coeff-GFCC-10',
 'Detail Coeff-MFCC-10',
 'Detail Coeff-GFCC-11',
 'Detail Coeff-MFCC-11',
 'Detail Coeff-GFCC-12',
 'Detail Coeff-MFCC-12',
 'Detail Coeff-GFCC-13',
 'Detail Coeff-MFCC-13',
 'Disease'
     ]

In [46]:
keys=list(feat.keys())
df = pd.DataFrame(columns=keys, index=[1])

start1=datetime.now()
i=1

for filename in glob.glob("CollectedDataset/Bron/*.wav"):
    start=datetime.now()

    patient_id=filename.split("\\")[1].split(".")[0].split("_")[0]

    y,sr=lib.load(filename, sr=4000)

    feat= get_overall_features(y)
    
    feat['Disease']="Bronchiectasis"
    df.loc[i]=feat
    
    print(filename)
    print("Time taken: "+str(datetime.now()-start))
    print("Time taken from start: "+str(datetime.now()-start1))
    df.to_csv("Features of Collected Data/Bron/Features_bron "+str(i)+ ".csv")
    i=i+1
    print()
    print()

print("Total time taken: "+str(datetime.now()-start1))

CollectedDataset/Bron\12_PAL_CR_BR.wav
Time taken: 0:00:36.714342
Time taken from start: 0:00:36.715338


CollectedDataset/Bron\12_PBL_CR_BR.wav
Time taken: 0:00:35.912970
Time taken from start: 0:01:12.718069


CollectedDataset/Bron\12_PML_CR_BR.wav
Time taken: 0:00:35.852445
Time taken from start: 0:01:48.639331


CollectedDataset/Bron\17_PAL_CR_BR.wav
Time taken: 0:00:36.316753
Time taken from start: 0:02:25.326773


CollectedDataset/Bron\17_PAR_CR_BR.wav
Time taken: 0:00:36.306956
Time taken from start: 0:03:01.977277


CollectedDataset/Bron\17_PBR_CR_BR.wav
Time taken: 0:00:36.184846
Time taken from start: 0:03:38.529799


CollectedDataset/Bron\17_PMB_CR_BR.wav
Time taken: 0:00:35.634529
Time taken from start: 0:04:14.224169


CollectedDataset/Bron\17_PML_CR_BR.wav
Time taken: 0:00:25.401080
Time taken from start: 0:04:39.638214


CollectedDataset/Bron\17_PMR_CR_BR.wav
Time taken: 0:00:37.617786
Time taken from start: 0:05:17.263988


CollectedDataset/Bron\19_PBL_CR_BR.wav
Time ta

In [49]:
keys=list(feat.keys())
df = pd.DataFrame(columns=keys, index=[1])

start1=datetime.now()
i=1

for filename in glob.glob("CollectedDataset/Asth/*.wav"):
    start=datetime.now()

    patient_id=filename.split("\\")[1].split(".")[0].split("_")[0]

    y,sr=lib.load(filename, sr=4000)

    feat= get_overall_features(y)
    
    feat['Disease']="Asthma"
    df.loc[i]=feat
    
    print(filename)
    print("Time taken: "+str(datetime.now()-start))
    print("Time taken from start: "+str(datetime.now()-start1))
    df.to_csv("Features of Collected Data/Asth/Features_asth "+str(i)+ ".csv")
    i=i+1
    print()
    print()

print("Total time taken: "+str(datetime.now()-start1))

CollectedDataset/Asth\10_AAL_W_A.wav
Time taken: 0:01:00.395840
Time taken from start: 0:01:00.567982


CollectedDataset/Asth\10_PAL_W_A.wav
Time taken: 0:01:00.598385
Time taken from start: 0:02:01.272087


CollectedDataset/Asth\10_PAR_W_A.wav
Time taken: 0:00:53.259061
Time taken from start: 0:02:54.588992


CollectedDataset/Asth\10_PBL_W_A.wav
Time taken: 0:00:38.114053
Time taken from start: 0:03:32.773007


CollectedDataset/Asth\10_PBR_W_A.wav
Time taken: 0:00:35.884521
Time taken from start: 0:04:09.007375


CollectedDataset/Asth\10_PMR_W_A.wav
Time taken: 0:00:36.622006
Time taken from start: 0:04:45.639372


CollectedDataset/Asth\11_PAL_W_A.wav
Time taken: 0:00:36.290418
Time taken from start: 0:05:22.329789


CollectedDataset/Asth\11_PBL_W_A.wav
Time taken: 0:00:36.030769
Time taken from start: 0:05:58.420526


CollectedDataset/Asth\11_PBR_W_A.wav
Time taken: 0:00:11.533842
Time taken from start: 0:06:10.014338


CollectedDataset/Asth\11_PML_W_A.wav
Time taken: 0:00:36.150704


CollectedDataset/Asth\3_PAL_W_A.wav
Time taken: 0:00:01.819029
Time taken from start: 0:36:54.215766


CollectedDataset/Asth\3_PAR_W_A.wav
Time taken: 0:00:35.131251
Time taken from start: 0:37:29.386993


CollectedDataset/Asth\3_PMR_W_A.wav
Time taken: 0:00:02.848480
Time taken from start: 0:37:32.275452


CollectedDataset/Asth\41_AAR_W_A.wav
Time taken: 0:00:02.188835
Time taken from start: 0:37:34.484276


CollectedDataset/Asth\41_PAL_W_A.wav
Time taken: 0:00:35.011085
Time taken from start: 0:38:09.515348


CollectedDataset/Asth\41_PAR_W_A.wav
Time taken: 0:00:34.881384
Time taken from start: 0:38:44.426715


CollectedDataset/Asth\41_PML_W_A.wav
Time taken: 0:00:35.274186
Time taken from start: 0:39:19.740879


CollectedDataset/Asth\41_PMR_W_A.wav
Time taken: 0:00:03.638059
Time taken from start: 0:39:23.408922


CollectedDataset/Asth\43_AAR_W_A.wav
Time taken: 0:00:34.941350
Time taken from start: 0:39:58.380255


CollectedDataset/Asth\43_PAL_W_A.wav
Time taken: 0:00:34.641511
Tim

CollectedDataset/Asth\64_PBR_A.wav
Time taken: 0:00:23.810191
Time taken from start: 1:05:39.528459


CollectedDataset/Asth\64_PMR_A.wav
Time taken: 0:00:37.229829
Time taken from start: 1:06:16.798266


CollectedDataset/Asth\66_PAL_A.wav
Time taken: 0:00:41.816138
Time taken from start: 1:06:58.691285


CollectedDataset/Asth\66_PML_A.wav
Time taken: 0:00:03.135981
Time taken from start: 1:07:01.903064


CollectedDataset/Asth\69_PAL_W_A.wav
Time taken: 0:00:39.452888
Time taken from start: 1:07:41.392852


CollectedDataset/Asth\69_PAR_W_A.wav
Time taken: 0:00:44.695310
Time taken from start: 1:08:26.166442


CollectedDataset/Asth\69_PML_W_A.wav
Time taken: 0:00:41.243165
Time taken from start: 1:09:07.478929


CollectedDataset/Asth\69_PMR_W_A.wav
Time taken: 0:00:40.012180
Time taken from start: 1:09:47.565910


CollectedDataset/Asth\70_PAL_W_A.wav
Time taken: 0:00:41.704231
Time taken from start: 1:10:29.344892


CollectedDataset/Asth\70_PAR_W_A.wav
Time taken: 0:00:41.155124
Time tak

In [47]:
keys=list(feat.keys())
df = pd.DataFrame(columns=keys, index=[1])

start1=datetime.now()
i=1

for filename in glob.glob("CollectedDataset/COPD/*.wav"):
    start=datetime.now()

    patient_id=filename.split("\\")[1].split(".")[0].split("_")[0]

    y,sr=lib.load(filename, sr=4000)

    feat= get_overall_features(y)
    
    feat['Disease']="COPD"
    df.loc[i]=feat
    
    print(filename)
    print("Time taken: "+str(datetime.now()-start))
    print("Time taken from start: "+str(datetime.now()-start1))
    df.to_csv("Features of Collected Data/COPD/Features_COPD "+str(i)+ ".csv")
    i=i+1
    print()
    print()

print("Total time taken: "+str(datetime.now()-start1))

CollectedDataset/COPD\34_PAL_W_C.wav
Time taken: 0:00:03.671115
Time taken from start: 0:00:03.678201


CollectedDataset/COPD\34_PAR_W_C.wav
Time taken: 0:00:27.667019
Time taken from start: 0:00:31.347216


CollectedDataset/COPD\34_PMR_W_C.wav
Time taken: 0:00:02.558160
Time taken from start: 0:00:33.911361


CollectedDataset/COPD\36_AAR_W_C.wav
Time taken: 0:00:38.703822
Time taken from start: 0:01:12.619172


CollectedDataset/COPD\36_PAL_W_C.wav
Time taken: 0:00:37.427943
Time taken from start: 0:01:50.098977


CollectedDataset/COPD\36_PAR_W_C.wav
Time taken: 0:00:25.207724
Time taken from start: 0:02:15.391719


CollectedDataset/COPD\36_PBL_W_C.wav
Time taken: 0:00:29.556965
Time taken from start: 0:02:44.988578


CollectedDataset/COPD\36_PBR_W_C.wav
Time taken: 0:00:03.679164
Time taken from start: 0:02:48.674723


CollectedDataset/COPD\39_AAR_W_C.wav
Time taken: 0:00:36.093488
Time taken from start: 0:03:24.774195


CollectedDataset/COPD\39_PAL_W_C.wav
Time taken: 0:00:36.968149


In [48]:
keys=list(feat.keys())
df = pd.DataFrame(columns=keys, index=[1])

start1=datetime.now()
i=1

for filename in glob.glob("CollectedDataset/Hlth/*.wav"):
    start=datetime.now()

    patient_id=filename.split("\\")[1].split(".")[0].split("_")[0]

    y,sr=lib.load(filename, sr=4000)

    feat= get_overall_features(y)
    
    feat['Disease']="Healthy"
    df.loc[i]=feat
    
    print(filename)
    print("Time taken: "+str(datetime.now()-start))
    print("Time taken from start: "+str(datetime.now()-start1))
    df.to_csv("Features of Collected Data/Hlth/Features_hlth "+str(i)+ ".csv")
    i=i+1
    print()
    print()

print("Total time taken: "+str(datetime.now()-start1))

CollectedDataset/Hlth\102_1b1_Ar_sc_Meditron.wav
Time taken: 0:00:23.534832
Time taken from start: 0:00:23.571966


CollectedDataset/Hlth\121_1b1_Tc_sc_Meditron.wav
Time taken: 0:00:06.296891
Time taken from start: 0:00:29.948811


CollectedDataset/Hlth\121_1p1_Tc_sc_Meditron.wav
Time taken: 0:00:04.035577
Time taken from start: 0:00:34.034361


CollectedDataset/Hlth\123_1b1_Al_sc_Meditron.wav
Time taken: 0:00:48.909338
Time taken from start: 0:01:22.993672


CollectedDataset/Hlth\125_1b1_Tc_sc_Meditron.wav
Time taken: 0:00:39.311189
Time taken from start: 0:02:02.364855


CollectedDataset/Hlth\126_1b1_Al_sc_Meditron.wav
Time taken: 0:00:19.560867
Time taken from start: 0:02:22.015675


CollectedDataset/Hlth\127_1b1_Ar_sc_Meditron.wav
Time taken: 0:00:38.625182
Time taken from start: 0:03:00.680834


CollectedDataset/Hlth\136_1b1_Ar_sc_Meditron.wav
Time taken: 0:00:40.879001
Time taken from start: 0:03:41.613466


CollectedDataset/Hlth\143_1b1_Al_sc_Meditron.wav
Time taken: 0:00:38.557